In [1]:
%install_ext https://raw.githubusercontent.com/szeitlin/watermark/master/watermark.py

Installed watermark.py. To use it, type:
  %load_ext watermark


In [2]:
%load_ext watermark

In [6]:
%watermark -a "Samantha Zeitlin" -d -u -p ipython,python,pandas,lxml,seaborn,pytz

Samantha Zeitlin 
Last updated: 03/07/2015 

ipython 3.0.0
python 2.7.9
ipython 3.0.0
pandas 0.15.2
lxml 3.3.5
seaborn 0.5.1
pytz 2014.9


In [7]:
from lxml import objectify
import pandas
import pytz
from dateutil.parser import parse

In [8]:
def extract_nodes(root):

    StartTime = []
    TotalTimeSeconds = []
    DistanceMeters = []
    MaximumSpeed = []
    Calories = []

    AvgSpeedList = []

    Pos_count, Alt_count = 0,0
    posTime = []
    Lat = []
    Long = []
    altTime = []
    alt = [] #altitude
    dist = [] #distancemeters

    #idea to use altitude, position and distance meters to calculate more finely-grained speeds

    for a,act in enumerate(root.Activities.Activity):

        for lap in root.Activities.Activity[a].Lap:
            StartTime.append(lap.values()[0])
            TotalTimeSeconds.append(float(lap.TotalTimeSeconds.text))
            DistanceMeters.append(float(lap.DistanceMeters.text))
            MaximumSpeed.append(float(lap.MaximumSpeed.text))
            Calories.append(float(lap.Calories.text))

            kids= lap.Extensions.getchildren()
            avgspeed = kids[1].AvgSpeed.pyval #have to do pyval to get rid of nested brackets from xml
            AvgSpeedList.append(avgspeed)

            try:
                for j, lapnum in enumerate(lap):
                    if lap[j].Track is not None:
                        try:
                            if lap[j].Track.Trackpoint is not None:

                                for k, trackpt in enumerate(lap[j].Track.Trackpoint):
                                    try:
                                        if lap[j].Track.Trackpoint[k].Position is not None:
                                            Pos_count+=1
                                            posTime.append(lap[j].Track.Trackpoint[k].Time.pyval)
                                            Lat.append(lap[j].Track.Trackpoint[k].Position.LatitudeDegrees.pyval)
                                            Long.append(lap[j].Track.Trackpoint[k].Position.LongitudeDegrees.pyval)
                                    except AttributeError:
                                        continue
                                    try:
                                        if lap[j].Track.Trackpoint[k].AltitudeMeters.text is not None:
                                            Alt_count+=1
                                            altTime.append(lap[j].Track.Trackpoint[k].Time.pyval)
                                            alt.append(lap[j].Track.Trackpoint[k].AltitudeMeters.text)
                                            dist.append(lap[j].Track.Trackpoint[k].DistanceMeters.text)
                                    except AttributeError:
                                        continue
                        except AttributeError:
                            continue

            except AttributeError:
                continue

    all_the_lists = [StartTime, TotalTimeSeconds, DistanceMeters, MaximumSpeed, Calories, AvgSpeedList, posTime, Lat, Long, altTime, alt,dist]

    return all_the_lists

In [9]:
path = "2014.tcx"
parsed = objectify.parse(open(path))
root = parsed.getroot()
all_the_lists = extract_nodes(root)

In [10]:
print map(len, all_the_lists)

[523, 523, 523, 523, 523, 523, 12287, 12287, 12287, 12287, 12287, 12287]


In [11]:
seriously = map(pandas.Series, all_the_lists)

In [12]:
[item.dtypes for item in seriously]

[dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('O')]

In [13]:
nummier = [item.convert_objects(convert_numeric=True) for item in seriously]

In [14]:
sigfigs = [item.round(4) if item.dtype =='float64' else item for item in nummier ]

In [15]:
[item.dtypes for item in sigfigs]

[dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64')]

In [16]:
df= pandas.concat(sigfigs, axis=1)

In [36]:
df.shape

(12287, 12)

In [17]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,2014-02-07T14:41:03Z,614.76,4822.5229,12.425,275,7.8446,2014-10-16T14:48:11Z,37.3943,-122.0769,2014-10-16T14:48:11Z,19.3638,4780.8071
1,2014-02-07T15:47:50Z,1751.72,14645.6211,12.125,569,8.3607,2014-10-16T14:48:12Z,37.3943,-122.0771,2014-10-16T14:48:12Z,19.3638,4791.6362
2,2014-10-16T13:37:12Z,629.12,4780.8071,12.490,261,7.5992,2014-10-16T14:48:17Z,37.3944,-122.0774,2014-10-16T14:48:17Z,19.8444,4826.5288
3,2014-10-16T14:48:11Z,1594.29,14645.1543,14.130,595,9.1860,2014-10-16T14:48:26Z,37.3947,-122.0781,2014-10-16T14:48:26Z,19.8444,4895.5474
4,2014-09-16T23:23:18Z,1629.11,14082.7471,12.798,573,8.6444,2014-10-16T14:48:29Z,37.3948,-122.0782,2014-10-16T14:48:29Z,19.8444,4916.4795


In [18]:
df.columns = ['StartTime', 'TotalTimeSeconds', 'TotalDistanceMeters', 'MaximumSpeed', 'Calories', 'AvgSpeed', 'posTime', 'Lat', 'Long', 'altTime', 'Altitude','DistanceMeters']

In [19]:
df.head()

,StartTime,TotalTimeSeconds,TotalDistanceMeters,MaximumSpeed,Calories,AvgSpeed,posTime,Lat,Long,altTime,Altitude,DistanceMeters
0,2014-02-07T14:41:03Z,614.76,4822.5229,12.425,275,7.8446,2014-10-16T14:48:11Z,37.3943,-122.0769,2014-10-16T14:48:11Z,19.3638,4780.8071
1,2014-02-07T15:47:50Z,1751.72,14645.6211,12.125,569,8.3607,2014-10-16T14:48:12Z,37.3943,-122.0771,2014-10-16T14:48:12Z,19.3638,4791.6362
2,2014-10-16T13:37:12Z,629.12,4780.8071,12.490,261,7.5992,2014-10-16T14:48:17Z,37.3944,-122.0774,2014-10-16T14:48:17Z,19.8444,4826.5288
3,2014-10-16T14:48:11Z,1594.29,14645.1543,14.130,595,9.1860,2014-10-16T14:48:26Z,37.3947,-122.0781,2014-10-16T14:48:26Z,19.8444,4895.5474
4,2014-09-16T23:23:18Z,1629.11,14082.7471,12.798,573,8.6444,2014-10-16T14:48:29Z,37.3948,-122.0782,2014-10-16T14:48:29Z,19.8444,4916.4795


In [20]:
#will have to match up StartTime with posTime and altTime, sort and filter accordingly.
#actually makes more sense to make them separate, check to make sure totals match, and then join back together later. 
#want to split without just making a view

In [38]:
df.to_csv("2014_df_unsorted.csv",index_col=0)

In [21]:
ts = df.loc[:,['posTime','Lat', 'Long','altTime', 'Altitude', 'DistanceMeters']]

In [22]:
ts.head()

,posTime,Lat,Long,altTime,Altitude,DistanceMeters
0,2014-10-16T14:48:11Z,37.3943,-122.0769,2014-10-16T14:48:11Z,19.3638,4780.8071
1,2014-10-16T14:48:12Z,37.3943,-122.0771,2014-10-16T14:48:12Z,19.3638,4791.6362
2,2014-10-16T14:48:17Z,37.3944,-122.0774,2014-10-16T14:48:17Z,19.8444,4826.5288
3,2014-10-16T14:48:26Z,37.3947,-122.0781,2014-10-16T14:48:26Z,19.8444,4895.5474
4,2014-10-16T14:48:29Z,37.3948,-122.0782,2014-10-16T14:48:29Z,19.8444,4916.4795


In [23]:
#have to convert datetimes to the right timezone - maybe make this a reusable utility ! 

In [24]:
ts['posTime']=[parse(x) for x in ts['posTime']]

In [25]:
ts['altTime']=[parse(x) for x in ts['altTime']]

In [26]:
ts['posTime']=[x.astimezone(pytz.timezone('US/Pacific')) for x in ts['posTime']]

In [27]:
ts['altTime']=[x.astimezone(pytz.timezone('US/Pacific')) for x in ts['altTime']]

In [71]:
#would be nice to have a way to do that to 2 columns at a time...!

In [28]:
ts.head()

,posTime,Lat,Long,altTime,Altitude,DistanceMeters
0,2014-10-16 07:48:11-07:00,37.3943,-122.0769,2014-10-16 07:48:11-07:00,19.3638,4780.8071
1,2014-10-16 07:48:12-07:00,37.3943,-122.0771,2014-10-16 07:48:12-07:00,19.3638,4791.6362
2,2014-10-16 07:48:17-07:00,37.3944,-122.0774,2014-10-16 07:48:17-07:00,19.8444,4826.5288
3,2014-10-16 07:48:26-07:00,37.3947,-122.0781,2014-10-16 07:48:26-07:00,19.8444,4895.5474
4,2014-10-16 07:48:29-07:00,37.3948,-122.0782,2014-10-16 07:48:29-07:00,19.8444,4916.4795


In [29]:
#want to group by day, maybe? 

In [30]:
#test whether posTime and altTime are identical
ident = ts['posTime']==ts['altTime']

In [31]:
["no" for item in ident if item==False]

[]

In [81]:
#['Yes' for item in ident if item==True] #just a test to confirm line above gives the right result

In [32]:
ts['weekday']=[item.weekday() for item in ts.posTime]

In [39]:
ts.head()

,posTime,Lat,Long,altTime,Altitude,DistanceMeters,weekday
0,2014-10-16 07:48:11-07:00,37.3943,-122.0769,2014-10-16 07:48:11-07:00,19.3638,4780.8071,3
1,2014-10-16 07:48:12-07:00,37.3943,-122.0771,2014-10-16 07:48:12-07:00,19.3638,4791.6362,3
2,2014-10-16 07:48:17-07:00,37.3944,-122.0774,2014-10-16 07:48:17-07:00,19.8444,4826.5288,3
3,2014-10-16 07:48:26-07:00,37.3947,-122.0781,2014-10-16 07:48:26-07:00,19.8444,4895.5474,3
4,2014-10-16 07:48:29-07:00,37.3948,-122.0782,2014-10-16 07:48:29-07:00,19.8444,4916.4795,3


In [40]:
ts.to_csv("2014_ts.csv",index_col=0)

In [33]:
daily = ts.groupby('weekday')

In [34]:
daily.describe()

Altitude  DistanceMeters          Lat         Long
weekday                                                             
0       count  2119.000000     2119.000000  2119.000000  2119.000000
        mean     21.973189     7228.883185    37.513257  -122.148099
        std      16.485939     5127.249537     0.178643     0.186464
        min     -15.243700        0.000000    37.374600  -122.433100
        25%      10.711800     1998.694000    37.385650  -122.391700
        50%      19.844400     6547.623000    37.394400  -122.060500
        75%      30.899500    11840.889650    37.760900  -122.003500
        max      54.451800    16621.033200    37.778700  -121.950400
1       count  3142.000000     3142.000000  3142.000000  3142.000000
        mean     19.836098     6810.421778    37.494161  -122.130365
        std       9.074371     4698.481629     0.171915     0.179832
        min      -5.149900        0.000000    37.374600  -122.433400
        25%      13.115100     2058.318975    37.382825  -122.391500
        50%      18.883100     6352.022000    37.391500  -122.050100
        75%      26.092900    10634.279050    37.758875  -122.001525
        max      50.126000    15592.120100    37.778600  -121.950600
2       count  2261.000000     2261.000000  2261.000000  2261.000000
        mean     21.753181     7840.630873    37.469225  -122.098310
        std      16.440591     5139.973861     0.156795     0.165662
        min     -11.879200        0.000000    37.373400  -122.433400
        25%      10.231200     3383.466600    37.382900  -122.077600
        50%      20.805700     7187.048300    37.390100  -122.044000
        75%      29.457500    12662.295900    37.395900  -121.969700
        max      53.490500    17944.203100    37.778600  -121.950300
3       count  2630.000000     2630.000000  2630.000000  2630.000000
        mean     16.356567    10363.626709    37.426075  -122.044358
        std       8.884694     5429.846219     0.119263     0.130741
        min       2.060100      129.355900    37.373400  -122.428500
        25%      10.711800     6041.589375    37.377700  -122.049500
        50%      14.557000    10658.839800    37.385800  -121.999800
        75%      21.286400    14965.998250    37.394300  -121.968800
        max      51.567900    19425.960900    37.778900  -121.950400
4       count  2135.000000     2135.000000  2135.000000  2135.000000
        mean     14.367773     9924.902202    37.468817  -122.097581
        std       9.039924     6618.133557     0.156138     0.165199
        min      -8.514600      230.290200    37.373400  -122.430600
        25%       7.827900     5047.648900    37.382000  -122.077700
        50%      15.037700     9054.567400    37.390200  -122.039400
        75%      21.767000    14484.263700    37.396000  -121.976100
        max      31.860700    28266.460900    37.776600  -121.950500

In [88]:
#to do: max and mean altitude change per day, calculate speeds, map routes?
#would be great to be able to mouse over the route and show speed, altitude

In [89]:
#would be nice to be able to do "for group in groupby"

In [90]:
#also would be nice to not lose the actual dates on these

In [92]:
ts.head()

,posTime,Lat,Long,altTime,Altitude,DistanceMeters,day
0,2014-10-16 07:48:11-07:00,37.3943,-122.0769,2014-10-16 07:48:11-07:00,19.3638,4780.8071,16
1,2014-10-16 07:48:12-07:00,37.3943,-122.0771,2014-10-16 07:48:12-07:00,19.3638,4791.6362,16
2,2014-10-16 07:48:17-07:00,37.3944,-122.0774,2014-10-16 07:48:17-07:00,19.8444,4826.5288,16
3,2014-10-16 07:48:26-07:00,37.3947,-122.0781,2014-10-16 07:48:26-07:00,19.8444,4895.5474,16
4,2014-10-16 07:48:29-07:00,37.3948,-122.0782,2014-10-16 07:48:29-07:00,19.8444,4916.4795,16
